In [1]:
# All imports required
import os
import urllib3
import numpy as np
import pandas as pd
import itertools
import matplotlib.pyplot as plt
from scipy.stats import norm, multivariate_normal
from sklearn.model_selection import train_test_split

In [2]:
# Check if dataset is already present in project directory, else download from UCI repository
if not os.path.exists('wine.data'):
    http = urllib3.PoolManager()
    url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data'
    r = http.request('GET', url)
    with open('wine.data', 'wb') as f:
        f.write(r.data)
        r.status

In [3]:
# Load dataset
arr = np.loadtxt('wine.data', delimiter=',')
arr[1:5]

array([[1.000e+00, 1.320e+01, 1.780e+00, 2.140e+00, 1.120e+01, 1.000e+02,
        2.650e+00, 2.760e+00, 2.600e-01, 1.280e+00, 4.380e+00, 1.050e+00,
        3.400e+00, 1.050e+03],
       [1.000e+00, 1.316e+01, 2.360e+00, 2.670e+00, 1.860e+01, 1.010e+02,
        2.800e+00, 3.240e+00, 3.000e-01, 2.810e+00, 5.680e+00, 1.030e+00,
        3.170e+00, 1.185e+03],
       [1.000e+00, 1.437e+01, 1.950e+00, 2.500e+00, 1.680e+01, 1.130e+02,
        3.850e+00, 3.490e+00, 2.400e-01, 2.180e+00, 7.800e+00, 8.600e-01,
        3.450e+00, 1.480e+03],
       [1.000e+00, 1.324e+01, 2.590e+00, 2.870e+00, 2.100e+01, 1.180e+02,
        2.800e+00, 2.690e+00, 3.900e-01, 1.820e+00, 4.320e+00, 1.040e+00,
        2.930e+00, 7.350e+02]])

In [4]:
# Read array as a dataframe
feature_names = ['Winery', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols',
                 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue',
                 'OD280/OD315 of diluted wines', 'Proline']
df = pd.DataFrame(arr, columns=feature_names)
df.describe()

,Winery,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
count,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000,178.000000
mean,1.938202,13.000618,2.336348,2.366517,19.494944,99.741573,2.295112,2.029270,0.361854,1.590899,5.058090,0.957449,2.611685,746.893258
std,0.775035,0.811827,1.117146,0.274344,3.339564,14.282484,0.625851,0.998859,0.124453,0.572359,2.318286,0.228572,0.709990,314.907474
min,1.000000,11.030000,0.740000,1.360000,10.600000,70.000000,0.980000,0.340000,0.130000,0.410000,1.280000,0.480000,1.270000,278.000000
25%,1.000000,12.362500,1.602500,2.210000,17.200000,88.000000,1.742500,1.205000,0.270000,1.250000,3.220000,0.782500,1.937500,500.500000
50%,2.000000,13.050000,1.865000,2.360000,19.500000,98.000000,2.355000,2.135000,0.340000,1.555000,4.690000,0.965000,2.780000,673.500000
75%,3.000000,13.677500,3.082500,2.557500,21.500000,107.000000,2.800000,2.875000,0.437500,1.950000,6.200000,1.120000,3.170000,985.000000
max,3.000000,14.830000,5.800000,3.230000,30.000000,162.000000,3.880000,5.080000,0.660000,3.580000,13.000000,1.710000,4.000000,1680.000000


In [5]:
# Split data and target into training sets and test sets
X, y = arr[:,1:], arr[:,0]
X_train, X_test, y_train, y_test = train_test_split(X, y)
print('Dimensions of X_train: {}'.format(X_train.shape))
print('Dimensions of X_test: {}'.format(X_test.shape))
print('Dimensions of y_train: {}'.format(y_train.shape))
print('Dimensions of y_test: {}'.format(y_test.shape))

Dimensions of X_train: (133, 13)
Dimensions of X_test: (45, 13)
Dimensions of y_train: (133,)
Dimensions of y_test: (45,)


In [6]:
# Fit a Gaussian generative model to the training data
classes = 3
features = X_train.shape[1]
mu = np.zeros((classes, features))
sigma = np.zeros((classes, features, features))
pi = np.zeros((classes))
for label in range(classes):
    indexes = y_train == label + 1
    mu[label] = np.mean(X_train[indexes], axis=0)
    sigma[label] = np.cov(X_train[indexes], rowvar=False, bias=True)
    pi[label] = np.mean(indexes)
pi.shape

(3,)

In [7]:
# Make predictions on test set
prediction_scores = np.zeros((y_test.size, 3))
for row_index in range(y_test.size):
    for class_index in range(classes):
        prediction_scores[row_index, class_index] = \
        np.log(pi[class_index]) + multivariate_normal.logpdf(X_test[row_index], mean=mu[class_index], 
                                                             cov=sigma[class_index])
predictions = prediction_scores.argmax(axis=1) + 1

In [8]:
# Calculate prediction accuracy
np.mean(predictions == y_test) * 100

100.0